In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.8 MB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 347 kB 30.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 37.7 MB/s eta 0:00:01
     |████████████████████████████████| 509 kB 53.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.8 MB 4.1 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 4.7 MB 41.5 MB/s eta 0:00:01     |███████████▎                    | 1.6 MB 41.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 19.1 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 21.8 MB/s eta 0:00:01
     |████████████████████████████████| 249 kB 22.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
accounts = pd.read_csv("accounts.csv")
data_dict = pd.read_csv("data_dictionary.csv")
products = pd.read_csv("products.csv")
sales_pipeline = pd.read_csv("sales_pipeline.csv")
sales_teams = pd.read_csv("sales_teams.csv")

In [ ]:
# inspect data
for name, df in {
    "accounts": accounts,
    "products": products,
    "sales_pipeline": sales_pipeline,
    "sales_teams": sales_teams
}.items():
    print(f"\n--- {name.upper()} ---")
    print(df.info())
    print(df.head())


--- ACCOUNTS ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.8+ KB
None
            account     sector  year_established  revenue  employees  \
0  Acme Corporation  technolgy              1996  1100.04       2822   
1        Betasoloin    medical              1999   251.41        495   
2          Betatech    medical              1986   647.18       1185   
3        Bioholding    medical              2012   587.34       1356   
4           Bioplex    medical

In [ ]:
# standardize column names
def clean_columns(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
    return df

accounts = clean_columns(accounts)
products = clean_columns(products)
sales_pipeline = clean_columns(sales_pipeline)
sales_teams = clean_columns(sales_teams)

In [ ]:
# handle duplicates
accounts.drop_duplicates(inplace=True)
products.drop_duplicates(inplace=True)
sales_pipeline.drop_duplicates(inplace=True)
sales_teams.drop_duplicates(inplace=True)

In [ ]:
# handle missing values
def fill_missing(df):
    for col in df.columns:
        if df[col].dtype in ["int64", "float64"]:
            df[col].fillna(df[col].median(), inplace=True)
        else:
            df[col].fillna(df[col].mode()[0], inplace=True)
    return df

accounts = fill_missing(accounts)
products = fill_missing(products)
sales_pipeline = fill_missing(sales_pipeline)
sales_teams = fill_missing(sales_teams)

/var/folders/zv/ft654ndd6nb8khhg3y2nqnth0000gn/T/ipykernel_44453/4282500350.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/var/folders/zv/ft654ndd6nb8khhg3y2nqnth0000gn/T/ipykernel_44453/4282500350.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [11]:
# fix date columns
date_cols = [col for col in sales_pipeline.columns if "date" in col]
for col in date_cols:
    sales_pipeline[col] = pd.to_datetime(sales_pipeline[col], errors="coerce")

In [12]:
# normalize currency columns
currency_cols = [col for col in sales_pipeline.columns if "amount" in col or "revenue" in col]
for col in currency_cols:
    sales_pipeline[col] = (
        sales_pipeline[col]
        .astype(str)
        .str.replace(r"[$,]", "", regex=True)
        .astype(float)
    )

In [13]:
# standardize categorical values
cat_cols = [col for col in accounts.columns if accounts[col].dtype == "object"]
for col in cat_cols:
    accounts[col] = accounts[col].str.strip().str.title()

In [14]:
# save cleaned datasets
accounts.to_csv("cleaned_accounts.csv", index=False)
products.to_csv("cleaned_products.csv", index=False)
sales_pipeline.to_csv("cleaned_sales_pipeline.csv", index=False)
sales_teams.to_csv("cleaned_sales_teams.csv", index=False)

In [15]:
# check for duplicates
print("Accounts duplicates:", accounts.duplicated().sum())
print("Products duplicates:", products.duplicated().sum())
print("Pipeline duplicates:", sales_pipeline.duplicated().sum())
print("Teams duplicates:", sales_teams.duplicated().sum())

Accounts duplicates: 0
Products duplicates: 0
Pipeline duplicates: 0
Teams duplicates: 0


In [16]:
# check for missing values
print("Missing values per dataset:")
for name, df in {
    "accounts": accounts,
    "products": products,
    "sales_pipeline": sales_pipeline,
    "sales_teams": sales_teams
}.items():
    print(f"\n{name.upper()}")
    print(df.isnull().sum())

Missing values per dataset:

ACCOUNTS
account             0
sector              0
year_established    0
revenue             0
employees           0
office_location     0
subsidiary_of       0
dtype: int64

PRODUCTS
product        0
series         0
sales_price    0
dtype: int64

SALES_PIPELINE
opportunity_id    0
sales_agent       0
product           0
account           0
deal_stage        0
engage_date       0
close_date        0
close_value       0
dtype: int64

SALES_TEAMS
sales_agent        0
manager            0
regional_office    0
dtype: int64


In [17]:
# check data types
print(sales_pipeline.dtypes)

opportunity_id            object
sales_agent               object
product                   object
account                   object
deal_stage                object
engage_date       datetime64[ns]
close_date        datetime64[ns]
close_value              float64
dtype: object


In [18]:
# check categorical values
for col in accounts.select_dtypes(include="object").columns:
    print(f"\n{col}: {accounts[col].unique()[:10]}")


account: ['Acme Corporation' 'Betasoloin' 'Betatech' 'Bioholding' 'Bioplex'
 'Blackzim' 'Bluth Company' 'Bubba Gump' 'Cancity' 'Cheers']

sector: ['Technolgy' 'Medical' 'Retail' 'Software' 'Entertainment' 'Marketing'
 'Telecommunications' 'Finance' 'Employment' 'Services']

office_location: ['United States' 'Kenya' 'Philipines' 'Japan' 'Italy' 'Norway' 'Korea'
 'Jordan' 'Brazil' 'Germany']

subsidiary_of: ['Acme Corporation' 'Massive Dynamic' 'Bubba Gump' 'Inity' 'Sonron'
 'Golddex' 'Warephase']


In [19]:
# check numeric ranges
print(sales_pipeline.describe())

                         engage_date                     close_date  \
count                           8800                           8800   
mean   2017-06-16 11:54:55.636363520  2017-07-15 06:11:37.090909184   
min              2016-10-20 00:00:00            2017-03-01 00:00:00   
25%              2017-04-07 00:00:00            2017-05-22 00:00:00   
50%              2017-07-01 00:00:00            2017-06-13 00:00:00   
75%              2017-08-22 00:00:00            2017-09-20 00:00:00   
max              2017-12-27 00:00:00            2017-12-31 00:00:00   
std                              NaN                            NaN   

        close_value  
count   8800.000000  
mean    1249.038864  
min        0.000000  
25%        0.000000  
50%      472.000000  
75%     1085.250000  
max    30288.000000  
std     2072.411434  


In [ ]:
# data visualization to be made